In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [7]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool


search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=10,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    allow_delegation=True,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    allow_delegation=True,
    verbose=True,
)

In [8]:
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="ideas_task.md",
    human_input=True,
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,
    context=[
        brainstorm_task,
    ],
    output_file="selection_task.md",
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="research_task.md",
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="competitors_task.md",
)

inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="inspiration_task.md",
)

script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="script_task.md",
)

In [9]:
crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        industry="Hot Sauce",
    ),
)

print(result)

2024-10-10 18:51:02,972 - 14568 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Marketer
 [INFO]: == Starting Task: Come up with 5 video ideas for a Youtube channel in the Hot Sauce industry


> Entering new CrewAgentExecutor chain...
I need to generate creative and engaging video ideas for a YouTube channel focused on the hot sauce industry. The content should appeal to a young audience, be entertaining, and have the potential to go viral.

Action: Ask question to co-worker  
Action Input: {"question": "What are some trending challenges or formats on YouTube that could be adapted to showcase hot sauce in a fun and engaging way?", "context": "We are looking for video ideas for a YouTube channel in the hot sauce industry. The ideas need to appeal to a young audience and ideally should be based on popular trends or challenges that are currently gaining traction on YouTube."}   

I encountered an error while trying to use the tool. This was the error: 'NoneType' object has no attribute 'startswith'.
 Tool Ask question to co-worker a